# Sports League Optimization: Comparative Analysis of Algorithms

This notebook presents a comprehensive analysis of different optimization algorithms applied to the Sports League problem. We compare Hill Climbing, Simulated Annealing, and Genetic Algorithm approaches, analyzing their performance across multiple metrics.

## Table of Contents
1. [Problem Definition](#1-problem-definition)
2. [Experimental Setup](#2-experimental-setup)
3. [Algorithm Implementations](#3-algorithm-implementations)
4. [Performance Comparison](#4-performance-comparison)
5. [Statistical Analysis](#5-statistical-analysis)
6. [Conclusion](#6-conclusion)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy import stats
import random
from copy import deepcopy

# Import our custom modules
from solution import LeagueSolution, LeagueHillClimbingSolution, LeagueSASolution
from evolution import (
    hill_climbing, 
    simulated_annealing, 
    genetic_algorithm,
    # Mutation operators
    mutate_swap,
    mutate_swap_constrained,
    mutate_team_shift,
    mutate_targeted_player_exchange,
    mutate_shuffle_within_team_constrained,
    # Crossover operators
    crossover_one_point,
    crossover_one_point_prefer_valid,
    crossover_uniform,
    crossover_uniform_prefer_valid,
    # Selection operators
    selection_tournament,
    selection_tournament_variable_k,
    selection_ranking,
    selection_boltzmann
)

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

## 1. Problem Definition

### 1.1 Sports League Problem

The Sports League problem involves assigning players to teams while satisfying specific constraints and optimizing for team balance. The goal is to create teams with similar average skill levels.

**Formal Definition:**
- We have 35 players with different positions (GK, DEF, MID, FWD) and skill levels
- We need to assign these players to 5 teams (7 players per team)
- Each team must have exactly 1 GK, 2 DEF, 2 MID, and 2 FWD
- Each team's total salary must not exceed 750M €
- The objective is to minimize the standard deviation of average team skills

### 1.2 Solution Representation

We represent a solution as a list of team assignments for each player. For example, if `solution.repr[0] = 2`, it means player 0 is assigned to team 2.

**Search Space Size:**
- For 35 players and 5 teams, the theoretical search space is 5^35
- With constraints, the actual feasible search space is much smaller, but still extremely large

### 1.3 Fitness Function

The fitness function calculates the standard deviation of the average skill levels across all teams. A lower value indicates more balanced teams, which is our optimization goal.

For invalid solutions (those violating constraints), we return infinity to ensure they are never selected.

In [ ]:
# Load player data
players_df = pd.read_csv("players.csv", sep=";")
players_data = players_df.to_dict(orient="records")

# Display the player data
players_df

### 1.4 Data Analysis

Let's analyze the player data to understand the distribution of skills, positions, and salaries.

In [ ]:
# Analyze player positions
position_counts = players_df['Position'].value_counts()
print("Position distribution:")
print(position_counts)

# Analyze skill distribution by position
plt.figure(figsize=(12, 6))
sns.boxplot(x='Position', y='Skill', data=players_df)
plt.title('Skill Distribution by Position')
plt.grid(True)
plt.show()

# Analyze salary distribution by position
plt.figure(figsize=(12, 6))
sns.boxplot(x='Position', y='Salary', data=players_df)
plt.title('Salary Distribution by Position')
plt.grid(True)
plt.show()

# Correlation between skill and salary
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Skill', y='Salary', hue='Position', data=players_df)
plt.title('Correlation between Skill and Salary')
plt.grid(True)
plt.show()

## 2. Experimental Setup

### 2.1 Metrics for Comparison

To ensure a fair comparison between different algorithms, we'll track the following metrics:

1. **Solution Quality**: The fitness value (standard deviation of average team skills)
2. **Function Evaluations**: Number of fitness function calls
3. **Iterations**: Number of algorithm iterations
4. **Runtime**: Actual execution time in seconds

### 2.2 Algorithm Configurations

We'll test the following algorithm configurations:

In [ ]:
# Define algorithm configurations
configs = {
    # Hill Climbing configurations
    'HC_Standard': {
        'algorithm': 'Hill Climbing',
        'params': {
            'max_iterations': 500,
            'max_no_improvement': 100,
            'verbose': False
        }
    },
    
    # Simulated Annealing configurations
    'SA_Standard': {
        'algorithm': 'Simulated Annealing',
        'params': {
            'initial_temperature': 200.0,
            'cooling_rate': 0.95,
            'min_temperature': 1e-5,
            'iterations_per_temp': 20,
            'verbose': False
        }
    },
    
    # Genetic Algorithm configurations
    'GA_Tournament_OnePoint': {
        'algorithm': 'Genetic Algorithm',
        'params': {
            'population_size': 100,
            'max_generations': 50,
            'selection_operator': selection_tournament,
            'crossover_operator': crossover_one_point_prefer_valid,
            'crossover_rate': 0.8,
            'mutation_operator': mutate_swap_constrained,
            'mutation_rate': 0.1,
            'elitism': True,
            'elitism_size': 2,
            'verbose': False
        }
    },
    'GA_Ranking_Uniform': {
        'algorithm': 'Genetic Algorithm',
        'params': {
            'population_size': 100,
            'max_generations': 50,
            'selection_operator': selection_ranking,
            'crossover_operator': crossover_uniform_prefer_valid,
            'crossover_rate': 0.8,
            'mutation_operator': mutate_targeted_player_exchange,
            'mutation_rate': 0.1,
            'elitism': True,
            'elitism_size': 2,
            'verbose': False
        }
    },
    'GA_Boltzmann_TeamShift': {
        'algorithm': 'Genetic Algorithm',
        'params': {
            'population_size': 100,
            'max_generations': 50,
            'selection_operator': selection_boltzmann,
            'selection_params': {'temperature': 1.0},
            'crossover_operator': crossover_one_point_prefer_valid,
            'crossover_rate': 0.8,
            'mutation_operator': mutate_team_shift,
            'mutation_rate': 0.1,
            'elitism': True,
            'elitism_size': 2,
            'verbose': False
        }
    },
    'GA_Hybrid': {
        'algorithm': 'Hybrid GA',
        'params': {
            'population_size': 75,
            'max_generations': 40,
            'selection_operator': selection_tournament_variable_k,
            'selection_params': {'k': 3},
            'crossover_operator': crossover_uniform_prefer_valid,
            'crossover_rate': 0.85,
            'mutation_operator': mutate_shuffle_within_team_constrained,
            'mutation_rate': 0.15,
            'elitism': True,
            'elitism_size': 1,
            'local_search': {
                'algorithm': 'hill_climbing',
                'frequency': 5,  # Apply HC every 5 generations
                'iterations': 50  # HC iterations per application
            },
            'verbose': False
        }
    }
}

# Display the configurations
for name, config in configs.items():
    print(f"Configuration: {name}")
    print(f"Algorithm: {config['algorithm']}")
    print("Parameters:")
    for param, value in config['params'].items():
        if param not in ['selection_operator', 'crossover_operator', 'mutation_operator', 'verbose']:
            print(f"  {param}: {value}")
    print("")

### 2.3 Tracking Function Evaluations

To ensure fair comparison between algorithms, we'll implement a counter for fitness function evaluations:

In [ ]:
# Create a wrapper for the fitness function to count evaluations
class FitnessCounter:
    def __init__(self):
        self.count = 0
        self.original_fitness = LeagueSolution.fitness
        
    def start_counting(self):
        self.count = 0
        LeagueSolution.fitness = self.counting_fitness
        
    def stop_counting(self):
        LeagueSolution.fitness = self.original_fitness
        return self.count
    
    def counting_fitness(self, solution):
        self.count += 1
        return self.original_fitness(solution)

# Initialize the counter
fitness_counter = FitnessCounter()

### 2.4 Experiment Runner

We'll create a function to run experiments with all configurations and collect results:

In [ ]:
def run_experiments(configs, players_data, num_runs=5):
    results = []
    
    for config_name, config in configs.items():
        print(f"Running {config_name}...")
        
        for run in range(num_runs):
            # Reset random seed for each run to ensure fair comparison
            random.seed(42 + run)
            np.random.seed(42 + run)
            
            # Start counting fitness evaluations
            fitness_counter.start_counting()
            
            # Record start time
            start_time = time.time()
            
            # Run the appropriate algorithm
            if config['algorithm'] == 'Hill Climbing':
                # Create initial solution
                initial_solution = LeagueHillClimbingSolution(players=players_data)
                
                # Run Hill Climbing
                best_solution, best_fitness, history = hill_climbing(
                    initial_solution,
                    **config['params']
                )
                
                iterations = len(history)
                
            elif config['algorithm'] == 'Simulated Annealing':
                # Create initial solution
                initial_solution = LeagueSASolution(players=players_data)
                
                # Run Simulated Annealing
                best_solution, best_fitness, history = simulated_annealing(
                    initial_solution,
                    **config['params']
                )
                
                iterations = len(history)
                
            elif config['algorithm'] in ['Genetic Algorithm', 'Hybrid GA']:
                # Run Genetic Algorithm
                best_solution, best_fitness, history = genetic_algorithm(
                    players_data,
                    **config['params']
                )
                
                iterations = len(history)
            
            # Record end time and calculate runtime
            runtime = time.time() - start_time
            
            # Get number of fitness evaluations
            evaluations = fitness_counter.stop_counting()
            
            # Store results
            results.append({
                'Configuration': config_name,
                'Algorithm': config['algorithm'],
                'Run': run + 1,
                'Best Fitness': best_fitness,
                'Iterations': iterations,
                'Function Evaluations': evaluations,
                'Runtime (s)': runtime,
                'History': history
            })
            
            print(f"  Run {run + 1}: Fitness = {best_fitness:.6f}, Evaluations = {evaluations}, Runtime = {runtime:.2f}s")
    
    return pd.DataFrame(results)

## 3. Algorithm Implementations

### 3.1 Hill Climbing

Hill Climbing is a local search algorithm that starts with an initial solution and iteratively moves to better neighboring solutions until no improvement is possible.

**Key Components:**
- **Neighborhood Generation**: Defined in `LeagueHillClimbingSolution.get_neighbors()`, which generates valid neighboring solutions by swapping players between teams.
- **Selection Strategy**: We use steepest ascent, selecting the best neighbor at each iteration.
- **Termination Criteria**: The algorithm stops when no better neighbor is found or after a maximum number of iterations.

### 3.2 Simulated Annealing

Simulated Annealing is inspired by the annealing process in metallurgy. It allows accepting worse solutions with a probability that decreases over time, helping to escape local optima.

**Key Components:**
- **Random Neighbor Generation**: Defined in `LeagueSASolution.get_random_neighbor()`, which generates a random valid neighboring solution.
- **Acceptance Probability**: Based on the temperature and the fitness difference between the current and new solutions.
- **Cooling Schedule**: The temperature decreases over time, reducing the probability of accepting worse solutions.

### 3.3 Genetic Algorithm

Genetic Algorithm is a population-based search algorithm inspired by natural selection and genetics.

**Key Components:**
- **Selection Operators**: We've implemented three selection mechanisms:
  - Tournament Selection: Selects the best solution from k random candidates.
  - Ranking Selection: Selects solutions with probability proportional to their rank.
  - Boltzmann Selection: Uses Boltzmann distribution to select solutions.

- **Crossover Operators**: We've implemented three crossover operators:
  - One-Point Crossover: Creates a child by taking a portion from each parent.
  - One-Point Prefer Valid: Tries multiple cut points to find a valid solution.
  - Uniform Crossover: Creates a child by randomly selecting genes from either parent.

- **Mutation Operators**: We've implemented four mutation operators:
  - Swap: Randomly swaps two players between teams.
  - Swap Constrained: Swaps players of the same position.
  - Team Shift: Shifts all player assignments by a random number.
  - Targeted Player Exchange: Swaps players between teams to improve balance.
  - Shuffle Within Team: Shuffles players within a team with other teams.

- **Elitism**: Preserves the best solutions from one generation to the next.

### 3.4 Hybrid Approach

We've also implemented a hybrid approach that combines Genetic Algorithm with Hill Climbing, applying local search to the best solutions periodically.

## 4. Performance Comparison

Let's run the experiments and compare the performance of different algorithms:

In [ ]:
# Run experiments
results_df = run_experiments(configs, players_data, num_runs=3)

# Display summary statistics
summary = results_df.groupby('Configuration').agg({
    'Best Fitness': ['mean', 'std', 'min'],
    'Iterations': ['mean', 'std'],
    'Function Evaluations': ['mean', 'std'],
    'Runtime (s)': ['mean', 'std']
})

print("Summary Statistics:")
summary

### 4.1 Solution Quality Comparison

In [ ]:
# Compare solution quality (fitness)
plt.figure(figsize=(12, 6))
sns.boxplot(x='Configuration', y='Best Fitness', data=results_df)
plt.title('Solution Quality Comparison')
plt.ylabel('Fitness (lower is better)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

### 4.2 Computational Efficiency Comparison

In [ ]:
# Compare function evaluations
plt.figure(figsize=(12, 6))
sns.barplot(x='Configuration', y='Function Evaluations', data=results_df, estimator=np.mean, ci='sd')
plt.title('Function Evaluations Comparison')
plt.ylabel('Number of Function Evaluations')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# Compare runtime
plt.figure(figsize=(12, 6))
sns.barplot(x='Configuration', y='Runtime (s)', data=results_df, estimator=np.mean, ci='sd')
plt.title('Runtime Comparison')
plt.ylabel('Runtime (seconds)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

### 4.3 Convergence Comparison

In [ ]:
# Plot convergence curves
plt.figure(figsize=(14, 8))

# Get one history from each configuration (first run)
for config_name in configs.keys():
    history = results_df[results_df['Configuration'] == config_name].iloc[0]['History']
    plt.plot(history, label=config_name)

plt.title('Convergence Comparison')
plt.xlabel('Iterations')
plt.ylabel('Fitness (lower is better)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot normalized convergence curves (by function evaluations)
plt.figure(figsize=(14, 8))

for config_name in configs.keys():
    row = results_df[results_df['Configuration'] == config_name].iloc[0]
    history = row['History']
    evals = row['Function Evaluations']
    
    # Create x-axis as percentage of total evaluations
    x = np.linspace(0, 100, len(history))
    plt.plot(x, history, label=config_name)

plt.title('Normalized Convergence Comparison')
plt.xlabel('Percentage of Function Evaluations')
plt.ylabel('Fitness (lower is better)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 5. Statistical Analysis

### 5.1 ANOVA Test

Let's perform an ANOVA test to determine if there are statistically significant differences between the algorithms:

In [ ]:
# Perform ANOVA test on fitness values
groups = [results_df[results_df['Configuration'] == config]['Best Fitness'].values for config in configs.keys()]
f_stat, p_value = stats.f_oneway(*groups)

print(f"ANOVA Test Results:")
print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Statistically significant difference: {p_value < 0.05}")

# If ANOVA shows significant difference, perform post-hoc test
if p_value < 0.05:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    
    # Prepare data for Tukey's test
    fitness_values = results_df['Best Fitness'].values
    config_labels = results_df['Configuration'].values
    
    # Perform Tukey's test
    tukey_results = pairwise_tukeyhsd(fitness_values, config_labels, alpha=0.05)
    
    print("\nTukey's HSD Test Results:")
    print(tukey_results)

### 5.2 Efficiency vs. Quality Analysis

In [ ]:
# Create a scatter plot of fitness vs. function evaluations
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Function Evaluations', y='Best Fitness', hue='Configuration', data=results_df, s=100)
plt.title('Efficiency vs. Quality Analysis')
plt.xlabel('Number of Function Evaluations')
plt.ylabel('Fitness (lower is better)')
plt.grid(True)
plt.tight_layout()
plt.show()

# Create a scatter plot of fitness vs. runtime
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Runtime (s)', y='Best Fitness', hue='Configuration', data=results_df, s=100)
plt.title('Runtime vs. Quality Analysis')
plt.xlabel('Runtime (seconds)')
plt.ylabel('Fitness (lower is better)')
plt.grid(True)
plt.tight_layout()
plt.show()

## 6. Best Solution Analysis

Let's analyze the best solution found across all experiments:

In [ ]:
# Find the best overall solution
best_row = results_df.loc[results_df['Best Fitness'].idxmin()]
print(f"Best solution found by {best_row['Configuration']} (Run {best_row['Run']})")
print(f"Fitness: {best_row['Best Fitness']:.6f}")
print(f"Function Evaluations: {best_row['Function Evaluations']}")
print(f"Runtime: {best_row['Runtime (s)']:.2f} seconds")

# Re-run the best configuration to get the solution
config = configs[best_row['Configuration']]
random.seed(42 + best_row['Run'] - 1)
np.random.seed(42 + best_row['Run'] - 1)

if config['algorithm'] == 'Hill Climbing':
    initial_solution = LeagueHillClimbingSolution(players=players_data)
    best_solution, _, _ = hill_climbing(initial_solution, **config['params'])
elif config['algorithm'] == 'Simulated Annealing':
    initial_solution = LeagueSASolution(players=players_data)
    best_solution, _, _ = simulated_annealing(initial_solution, **config['params'])
else:  # Genetic Algorithm or Hybrid GA
    best_solution, _, _ = genetic_algorithm(players_data, **config['params'])

# Analyze the best solution
team_stats = best_solution.get_team_stats()

# Create a DataFrame for team statistics
teams_df = pd.DataFrame([
    {
        'Team': f"Team {stats['team_id']}",
        'Average Skill': stats['avg_skill'],
        'Total Salary': stats['total_salary'],
        'GK': stats['positions']['GK'],
        'DEF': stats['positions']['DEF'],
        'MID': stats['positions']['MID'],
        'FWD': stats['positions']['FWD']
    } for stats in team_stats
])

teams_df

In [ ]:
# Plot team skills
plt.figure(figsize=(10, 6))
bars = plt.bar(teams_df['Team'], teams_df['Average Skill'])
plt.title('Average Skill by Team')
plt.ylabel('Average Skill')
plt.axhline(y=teams_df['Average Skill'].mean(), color='r', linestyle='--', label='Mean')

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.2f}',
             ha='center', va='bottom')

plt.legend()
plt.grid(True)
plt.show()

# Plot team salaries
plt.figure(figsize=(10, 6))
bars = plt.bar(teams_df['Team'], teams_df['Total Salary'])
plt.title('Total Salary by Team')
plt.ylabel('Total Salary (M €)')
plt.axhline(y=750, color='r', linestyle='--', label='Budget Limit')

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{height:.1f}',
             ha='center', va='bottom')

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Display detailed team compositions
for i, stats in enumerate(team_stats):
    print(f"\nTeam {i}:")
    print(f"Average Skill: {stats['avg_skill']:.2f}")
    print(f"Total Salary: {stats['total_salary']} M €")
    print("Players:")
    
    # Create a DataFrame for this team's players
    team_df = pd.DataFrame(stats['players'])
    team_df = team_df.sort_values(by='Position')
    display(team_df)

## 7. Conclusion

### 7.1 Algorithm Performance Summary

Based on our experiments, we can draw the following conclusions:

1. **Solution Quality**: [To be filled after running experiments]
2. **Computational Efficiency**: [To be filled after running experiments]
3. **Convergence Behavior**: [To be filled after running experiments]

### 7.2 Operator Impact Analysis

1. **Selection Operators**: [To be filled after running experiments]
2. **Crossover Operators**: [To be filled after running experiments]
3. **Mutation Operators**: [To be filled after running experiments]
4. **Elitism Impact**: [To be filled after running experiments]

### 7.3 Recommendations

Based on our analysis, we recommend the following approach for solving the Sports League problem:

1. **Algorithm Choice**: [To be filled after running experiments]
2. **Parameter Settings**: [To be filled after running experiments]
3. **Operator Selection**: [To be filled after running experiments]

### 7.4 Future Work

Potential areas for future improvement include:

1. **Multi-objective Optimization**: Consider both team balance and other factors like player chemistry or strategic fit.
2. **Advanced Hybridization**: Explore more sophisticated combinations of global and local search.
3. **Adaptive Parameters**: Implement dynamic parameter adjustment based on search progress.
4. **Parallel Implementation**: Leverage parallel computing for population-based methods to improve efficiency.